## Final design of genelet OR gate.
<img align="left" src="Images/NORgate.png" alt="Genelet Logic Gate" width="900"/>

### Design
This genelet circuit is intended to solve the issue of the input genelets $\text{Sw}_1$ and $\text{Sw}_2$ being unable to produce the DNA activator strand $\text{A}_3$ in the initial design. This problem is avoided by $\text{Sw}_1$ and $\text{Sw}_2$ instead producing $\text{I}_3$, the inhibitor for $\text{Sw}_3$. There should be a sufficient quantity of activator strands in the mixture for all genelets. The inhibitor $\text{I}_3$ will sequester $\text{A}_3$ when it is produced and thus deactivate $\text{Sw}_3$. Since both switches $\text{Sw}_1$ and $\text{Sw}_2$ produce $\text{I}_3$, $\text{Sw}_3$ will always remain deactivated as long as at least one of the input switches are ON. Thus, the in order for $\text{Sw}_3$ to be activated both input switches need to be deactivated to stop $\text{I}_3$ production. When this happens, the enzyme RNAse H will degrade any present A-I complex and remove $\text{I}_3$ leaving behind only $\text{A}_3$ which in turn activates $\text{Sw}_3$. The behaviour of the first 3 genelet switches is thus that of a NOR gate. 

$\text{Sw}_4$ acts as a NOT gate to $\text{Sw}_3$, since the transcript of $\text{Sw}_3$ is $\text{I}_4$. Thus when $\text{Sw}_3$ is ON, $\text{I}_4$ is produced turning $\text{Sw}_4$ OFF and vice versa for when $\text{Sw}_3$ is OFF. A NOR gate paired with a NOT gate behaves as a regular OR gate.

### Code

In [35]:
# Imports

from biocrnpyler import *
import pylab as plt
import numpy as np
from bokeh.layouts import row

import warnings
import bokeh.io
import bokeh.plotting

# Mechanism class for Transcription Switch Component using complete reactions

class TranscriptionSwitch(Mechanism):
    """
    Reactions involved in this mechanism:
    
    Sw_OFF + A -> Sw_ON
    Sw_ON + I -> Sw_OFF + AI
    A + I -> AI
    Sw_OFF + RNAP <-> Sw_OFF.RNAP -> Sw_OFF + RNAP + P
    Sw_ON + RNAP <-> Sw_ON.RNAP -> Sw_ON + RNAP + P
    AI + RNAseH <-> AI.RNAseH -> A + RNAseH
    
    """
    
    # Set the name and mechanism_type
    def __init__(self, name="transcription_switch", mechanism_type="transcription"):
        
            Mechanism.__init__(self, name=name, mechanism_type=mechanism_type)
    
    def update_species(self, switch_off, activator, inhibitor, rnap, transcript, rnaseH, **keywords):

        return [switch_off, transcript, activator, inhibitor, rnap, rnaseH] 
    
    
    def update_reactions(self, switch_off, activator, inhibitor, component, part_id, rnap, rnaseH, transcript, **keywords):
        

        # Create complex species involved in mechanism
        switch_on = ComplexSpecies([switch_off, activator], name = str(switch_off).replace("_OFF","_ON"))
        A_I_complex = ComplexSpecies([inhibitor, activator],name = str(switch_off).replace("_OFF","")+"_AI")
        
        
        # Initialise reaction parameters
        ktx = component.get_parameter("ktx", part_id = part_id, mechanism = self)
        kleak = component.get_parameter("kleak", part_id = part_id, mechanism = self)
        kdeg = component.get_parameter("kdeg", part_id = part_id, mechanism = self)
        
        kb_tx = component.get_parameter("kb_tx", part_id = part_id, mechanism = self)
        kb_leak = component.get_parameter("kb_leak", part_id = part_id, mechanism = self)
        kb_deg = component.get_parameter("kb_deg", part_id = part_id, mechanism = self)
        
        kM_tx = component.get_parameter("kM_tx", part_id = part_id, mechanism = self)
        kM_leak = component.get_parameter("kM_leak", part_id = part_id, mechanism = self)
        kM_deg = component.get_parameter("kM_deg", part_id = part_id, mechanism = self)
        
        kon = component.get_parameter("kon", part_id = part_id, mechanism = self)
        koff = component.get_parameter("koff", part_id = part_id, mechanism = self)
        ka = component.get_parameter("ka", part_id = part_id, mechanism = self)
            
        #ku_tx = (kb_tx + ktx) / kM_tx
        #ku_leak = (kb_leak + kleak) / kM_leak
        #ku_deg = (kb_deg + kdeg) / kM_deg
        
        ku_tx = 0.1
        ku_leak = 0.1
        ku_deg = 0.1
        
        # Create reactions
        
        reaction_activation = Reaction(inputs = [switch_off, activator], outputs = [switch_on], 
                            k = kon )
        
        reaction_deactivation = Reaction(inputs = [switch_on, inhibitor], outputs = [switch_off, A_I_complex], 
                            k = koff )
        
        reaction_complex = Reaction(inputs = [activator, inhibitor], outputs = [A_I_complex], k = ka)
        
        # Step 1 of transcription and A-I degradation reactions
        
        reaction_tx_1 = Reaction(inputs = [switch_on, rnap], outputs = [ComplexSpecies([rnap, switch_on])], k = kb_tx, k_rev = ku_tx )
        
        reaction_leak_1 = Reaction(inputs = [switch_off, rnap], outputs = [ComplexSpecies([rnap, switch_off])], k = kb_leak, k_rev = ku_leak)
        
        reaction_deg_1 = Reaction(inputs = [A_I_complex, rnaseH], outputs = [ComplexSpecies([rnaseH, A_I_complex])], k = kb_deg, k_rev = ku_deg)
        
        # Step 2 of transcription and A-I degradation reactions
        
        reaction_tx_2 = Reaction(inputs = [ComplexSpecies([rnap, switch_on])], outputs = [switch_on, transcript, rnap], k = ktx)
        
        reaction_leak_2 = Reaction(inputs = [ComplexSpecies([rnap, switch_off])], outputs = [switch_off, transcript, rnap], k = kleak)
        
        reaction_deg_2 = Reaction(inputs = [ComplexSpecies([rnaseH, A_I_complex])], outputs = [rnaseH, activator], k = kdeg)
        
        return [reaction_activation, reaction_deactivation, reaction_complex, reaction_tx_1, reaction_tx_2, reaction_leak_1, reaction_leak_2, reaction_deg_1, reaction_deg_2]
    
    

In [36]:
class Genelet(Promoter):
    """
    Genelet switch component using TranscriptionSwitch() mechanism
    """
    def __init__(self, name, transcript, activator, inhibitor, rnap="RNAP",rnaseH="RNAseH" , **keywords):
        
        # Set the inputs
        # Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component.
        
        self.activator = self.set_species(activator, material_type = "dna") 
        self.inhibitor = self.set_species(inhibitor, material_type = "rna")
        self.transcript = self.set_species(transcript, material_type = "rna")
        self.switch_off = self.set_species(str(name)+"_OFF")
        self.rnap = self.set_species(rnap, material_type = "protein")
        self.rnaseH = self.set_species(rnaseH, material_type = "protein")
        
        custom_mechanisms = {"transcription": TranscriptionSwitch()}
        
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        
        mech_tx = self.mechanisms["transcription"]
        
        species = []
        species += mech_tx.update_species(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                          rnap = self.rnap, rnaseH = self.rnaseH)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = [] 
        reactions += mech_tx.update_reactions(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                              rnap = self.rnap, rnaseH = self.rnaseH, component = self, part_id = "Genelet", **keywords)
        return reactions

    

class Source(Promoter):
    """
    Genelet source component using Transcription_MM() mechanism
    """
    def __init__(self, name, transcript, rnap="RNAP", **keywords):
        
        # Set the inouts
        # Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component
        
        self.dna = self.set_species(name)
        self.rnap = self.set_species(rnap, material_type = "protein")
        
        custom_mechanisms = {"transcription": Transcription_MM()}
        
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        
        mech_tx = self.mechanisms["transcription"]
        
        species = [] 
        species += mech_tx.update_species(dna = self.dna, transcript = self.transcript, rnap = self.rnap)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = [] 
        reactions += mech_tx.update_reactions(dna = self.dna, transcript = self.transcript, 
                                              rnap = self.rnap, component = self, part_id = "Source", **keywords)
        return reactions    
    
    
    

In [37]:
# Creating CRN for NOR gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")
S4_off = Species("Sw4")

S1 = Genelet(S1_off, transcript = "I3", activator = "A1", inhibitor = "I1" )
S2 = Genelet(S2_off, transcript = "I3", activator = "A2", inhibitor = "I2" )
S3 = Genelet(S3_off, transcript = "I4", activator = "A3", inhibitor = "I3" )
#S4 = Genelet(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")

repr(M)
CRN = M.compile_crn()
print(CRN.pretty_print())

Species (12) = {0. Sw1_OFF, 1. rna[I3], 2. dna[A1], 3. rna[I1], 4. protein[RNAP], 5. protein[RNAseH], 6. Sw2_OFF, 7. dna[A2], 8. rna[I2], 9. Sw3_OFF, 10. rna[I4], 11. dna[A3]}
Reactions (27) = [
0. Sw1_OFF + dna[A1] --> complex[dna[A1]:Sw1_OFF]        
        massaction: k_f(Sw1_OFF,dna[A1])=0.0394*Sw1_OFF*dna[A1]
1. complex[dna[A1]:Sw1_OFF] + rna[I1] --> Sw1_OFF + complex[dna[A1]:rna[I1]]        
        massaction: k_f(complex[dna[A1]:Sw1_OFF],rna[I1])=0.696*complex[dna[A1]:Sw1_OFF]*rna[I1]
2. dna[A1] + rna[I1] --> complex[dna[A1]:rna[I1]]        
        massaction: k_f(dna[A1],rna[I1])=0.696*dna[A1]*rna[I1]
3. complex[dna[A1]:Sw1_OFF] + protein[RNAP] <--> complex[complex[dna[A1]:Sw1_OFF]:protein[RNAP]]        
        massaction: k_f(complex[dna[A1]:Sw1_OFF],protein[RNAP])=1.0*complex[dna[A1]:Sw1_OFF]*protein[RNAP]
        k_r(complex[complex[dna[A1]:Sw1_OFF]:protein[RNAP]])=0.1*complex[complex[dna[A1]:Sw1_OFF]:protein[RNAP]]
4. complex[complex[dna[A1]:Sw1_OFF]:protein[RNAP]] --> 

In [38]:
# Bioscrape simulation of NOR gate CRN

#io = {"Sw1_OFF": 2500, "dna_A1": 5000, "rna_I1": 0, "Sw2_OFF": 2500, "dna_A2": 0, "rna_I2": 0, "Sw3_OFF":700, "dna_A3": 1000, "rna_I3": 0, 
# "protein_RNAseH":100, "protein_RNAP":500} initial condition dictionary for concentration-dependent NAND gate

io = {"Sw1_OFF": 2500, "dna_A1": 2500, "rna_I1": 0, "Sw2_OFF": 2500, "dna_A2": 0, "rna_I2": 0, "Sw3_OFF":700, "dna_A3": 700,
      "rna_I3": 0, "protein_RNAseH":25, "protein_RNAP":500}

timepoints = np.linspace(0, 500, 1000)
R = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = io)

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
#s.circle(timepoints, R["complex_Sw2_AI"],legend_label = "AI complex 2", color = "green")

q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON"], legend_label = "ON switch 3" , color = "red")

#t = bokeh.plotting.figure(plot_width=300, plot_height=300)
#t.circle(timepoints, R["Sw4_OFF"], legend_label = "OFF switch 4", color = "orange")
#t.circle(timepoints, R["complex_Sw4_ON"], legend_label = "ON switch 4" , color = "red")

bokeh.io.show(row(p, s, q))
warnings.filterwarnings("ignore")

/home/nazmus/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.2.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: complex_Sw1_ON, complex_Sw1_AI, complex_Sw1_ON_protein_RNAP, complex_Sw1_OFF_protein_RNAP, complex_Sw1_AI_protein_RNAseH, complex_Sw2_ON, complex_Sw2_AI, complex_Sw2_ON_protein_RNAP, complex_Sw2_OFF_protein_RNAP, complex_Sw2_AI_protein_RNAseH, complex_Sw3_ON, complex_Sw3_AI, complex_Sw3_ON_protein_RNAP, complex_Sw3_OFF_protein_RNAP, complex_Sw3_AI_protein_RNAseH, 


Loading BokehJS ...

In [39]:
# NOR gate activators and inhibitors of importance

bokeh.io.output_notebook()

p = bokeh.plotting.figure(plot_width=400, plot_height=400)
p.circle(timepoints, R["dna_A3"], legend_label = "Activator 3", color = "orange")
p.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "red")
p.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 3", color = "blue")
#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")
p.legend.click_policy="hide"

s = bokeh.plotting.figure(plot_width=400, plot_height=400)
s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
#s.circle(timepoints, R["complex_Sw4_AI"], legend_label = "Complex 4", color = "blue")
s.legend.click_policy="hide"

bokeh.io.show(row(p, s))
warnings.filterwarnings("ignore")

Loading BokehJS ...